# Amazon Bedrock Reinforcement Fine-Tuning with OpenAI compatible APIs - Qwen3 32B

With OpenAI-compatible fine-tuning APIs, Amazon Bedrock can now help you train popular open-weight models, including OpenAI GPT-OSS and Qwen models using reinforcement fine-tuning (RFT).  


## What's RFT?

Traditional fine-tuning shows a model examples and says "produce outputs like this." RFT takes a different approach: it lets the model generate its own responses, then uses a reward signal to reinforce good outputs and discourage bad ones. Think of it like training with a coach who gives feedback rather than just copying from a textbook.

For math problems, this works particularly well because we can automatically verify if an answer is correct—no human labeling needed.

## What's GSM8K?

GSM8K (Grade School Math 8K) is a dataset of ~8,000 grade-school math word problems. Each problem requires multi-step reasoning to solve. It's become a standard benchmark for testing whether language models can actually "think" through problems rather than just pattern-match.

Example problem:
> *Janet's ducks lay 16 eggs per day. She eats three for breakfast and bakes muffins with four. She sells the rest at $2 each. How much does she make daily?*

This notebook demonstrates all fine-tuning API operations using the OpenAI SDK using the GSM8K dataset.

**API Operations Covered:**
1. Upload and list files
2. List submitted fine-tuning jobs
3. Create fine-tuning job (RFT)
4. Describe the submitted job
5. Optionally Cancel a job 
6. List events
7. List checkpoints

**Prerequisites:**
- SageMaker notebook with IAM role
- Training file `rft_train_data.jsonl` present in notebook

---
## Step 1: Install Required Dependencies

In [ ]:
%%capture install_output

# Install required packages
!pip install --upgrade boto3 botocore
!pip install --upgrade openai
!pip install --upgrade httpx
!pip install --upgrade colorama tiktoken aws-bedrock-token-generator

print("✅ Dependencies installed successfully!")

In [ ]:
# Verify installations
import boto3
import openai
import httpx


print(f"boto3 version: {boto3.__version__}")
print(f"openai version: {openai.__version__}")
print(f"httpx version: {httpx.__version__}")
print("\n✅ All imports successful! Tested with boto3 (v1.42.49), OpenAI (v2.21.0), httpx (v0.28.1)")

---
## Step 2: Configuration

### Bedrock API Keys

Before we can proceed, please use the following documentation to generate a short- or long-term Bedrock API Key:

https://docs.aws.amazon.com/bedrock/latest/userguide/api-keys.html

https://docs.aws.amazon.com/bedrock/latest/userguide/api-keys-generate.html#api-keys-generate-console

Here we will be using the Bedrock token generator library to create a shiort term key.

### Fine tuning role

Create a role (or edit your sagemaker execution role to have the following permissions):

- For Lambda invocation, the following shows an example policy you can use:

```
{
    "Version": "2012-10-17",		 	 	 
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": [
                "arn:aws:lambda:*:*:function:reward-function-name"
            ]
        }
    ]
}
```

- For RL using AI feedback, you will need to add specific permissions to invoke foundation models to the Lambda execution role. In your lambda role, you can configure these managed policies for LLMs for grading. See `AmazonBedrockLimitedAccess` .

The following is an example for invoking Amazon Bedrock foundation models as judge using the Invoke API:

```
{
    "Version": "2012-10-17",		 	 	 
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Resource": [
                "arn:aws:bedrock:*:*:foundation-model/*"
            ]
        }
    ]
}
```

Next, we will create the reward function required for our training the GPT-OSS model using RL.

# Lambda reward function creation

In [ ]:
import sys
import os
import json
import time

# Add the project root (two levels up) to the Python path so we can import helpers
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

from helpers import (
    create_lambda_deployment_package,
    cleanup_lambda_deployment_package
)

# ============== UPDATE THESE VALUES ==============
AWS_REGION = "us-west-2"
S3_BUCKET = "subshreyevals"
AWS_PROFILE = None  # Set to your profile name, or None for default credentials
# =================================================

# Create session
session = boto3.Session(profile_name=AWS_PROFILE, region_name=AWS_REGION) if AWS_PROFILE else boto3.Session(region_name=AWS_REGION)
AWS_ACCOUNT_ID = session.client('sts').get_caller_identity()['Account']

# Dataset configuration
DATASET_NAME = "gsm8k"
HF_DATASET = "openai/gsm8k"
LOCAL_DATA_DIR = "../../tmp-data"

assert S3_BUCKET != "your-bucket-name", "Please update S3_BUCKET with your own bucket name"

# S3 paths
S3_TRAINING_DATA = f"s3://{S3_BUCKET}/rft-data/datasets/{DATASET_NAME}/train.jsonl"
S3_VALIDATION_DATA = f"s3://{S3_BUCKET}/rft-data/datasets/{DATASET_NAME}/val.jsonl"
S3_OUTPUT_PATH = f"s3://{S3_BUCKET}/rft-output/"

# Resource names
LAMBDA_FUNCTION_NAME = f"{DATASET_NAME}-reward-function"
LAMBDA_ROLE_NAME = f"{DATASET_NAME.upper()}-Lambda-Role"
BEDROCK_ROLE_NAME = "BedrockRFTRole"
REWARD_FUNCTION_FILE = f"../../reward-functions/{DATASET_NAME}_gptoss_rew_func.py"
REWARD_FUNCTION_MODULE = f"{DATASET_NAME}_rew_func"

In [ ]:
# Create Lambda execution role
s3_client = session.client('s3', region_name="us-west-2")
bedrock_client = session.client('bedrock', region_name="us-west-2")
lambda_client = session.client('lambda', region_name="us-west-2")
iam_client = session.client('iam', region_name="us-west-2")

print("Creating Lambda execution role...")

lambda_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{"Effect": "Allow", "Principal": {"Service": "lambda.amazonaws.com"}, "Action": "sts:AssumeRole"}]
}

try:
    response = iam_client.create_role(
        RoleName=LAMBDA_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(lambda_trust_policy),
        Description=f"Execution role for {DATASET_NAME} reward function"
    )
    lambda_role_arn = response['Role']['Arn']
    iam_client.attach_role_policy(RoleName=LAMBDA_ROLE_NAME, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')
    print(f"✓ Created role: {LAMBDA_ROLE_NAME}")
    print("Waiting 10s for role propagation...")
    time.sleep(10)
except iam_client.exceptions.EntityAlreadyExistsException:
    lambda_role_arn = iam_client.get_role(RoleName=LAMBDA_ROLE_NAME)['Role']['Arn']
    print(f"✓ Using existing role: {LAMBDA_ROLE_NAME}")

# Package and deploy Lambda
lambda_zip_content = create_lambda_deployment_package(
    source_file=REWARD_FUNCTION_FILE,
    zip_filename="lambda_deployment.zip",
    archive_name=f"{REWARD_FUNCTION_MODULE}.py"
)

print(f"\nDeploying Lambda: {LAMBDA_FUNCTION_NAME}...")
try:
    lambda_client.get_function(FunctionName=LAMBDA_FUNCTION_NAME)
    lambda_client.update_function_code(FunctionName=LAMBDA_FUNCTION_NAME, ZipFile=lambda_zip_content)
    waiter = lambda_client.get_waiter('function_updated_v2')
    waiter.wait(FunctionName=LAMBDA_FUNCTION_NAME)
    print("✓ Updated existing function")
except lambda_client.exceptions.ResourceNotFoundException:
    lambda_client.create_function(
        FunctionName=LAMBDA_FUNCTION_NAME,
        Runtime='python3.11',
        Role=lambda_role_arn,
        Handler=f"{REWARD_FUNCTION_MODULE}.lambda_handler",
        Code={'ZipFile': lambda_zip_content},
        Timeout=300,
        MemorySize=512
    )
    print("✓ Created new function")

waiter = lambda_client.get_waiter('function_active_v2')
waiter.wait(FunctionName=LAMBDA_FUNCTION_NAME)
lambda_arn = lambda_client.get_function(FunctionName=LAMBDA_FUNCTION_NAME)['Configuration']['FunctionArn']
print(f"✓ Lambda ready: {lambda_arn}")

# Create Bedrock role
print(f"\nCreating Bedrock role: {BEDROCK_ROLE_NAME}...")

bedrock_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [{"Effect": "Allow", "Principal": {"Service": "bedrock.amazonaws.com"}, "Action": "sts:AssumeRole"}]
}

bedrock_permissions = {
    "Version": "2012-10-17",
    "Statement": [
        {"Effect": "Allow", "Action": ["s3:GetObject", "s3:ListBucket"], "Resource": [f"arn:aws:s3:::{S3_BUCKET}/*", f"arn:aws:s3:::{S3_BUCKET}"]},
        {"Effect": "Allow", "Action": "s3:PutObject", "Resource": f"arn:aws:s3:::{S3_BUCKET}/rft-output/*"},
        {"Effect": "Allow", "Action": "lambda:InvokeFunction", "Resource": lambda_arn}
    ]
}

try:
    response = iam_client.create_role(
        RoleName=BEDROCK_ROLE_NAME,
        AssumeRolePolicyDocument=json.dumps(bedrock_trust_policy),
        Description="Execution role for Bedrock RFT"
    )
    bedrock_role_arn = response['Role']['Arn']
    print(f"✓ Created role: {BEDROCK_ROLE_NAME}")
except iam_client.exceptions.EntityAlreadyExistsException:
    bedrock_role_arn = iam_client.get_role(RoleName=BEDROCK_ROLE_NAME)['Role']['Arn']
    print(f"✓ Using existing role: {BEDROCK_ROLE_NAME}")

iam_client.put_role_policy(RoleName=BEDROCK_ROLE_NAME, PolicyName='BedrockRFTPermissions', PolicyDocument=json.dumps(bedrock_permissions))
print(f"✓ Bedrock role ready: {bedrock_role_arn}")

cleanup_lambda_deployment_package()

In [ ]:
lambda_arn

In [ ]:
TARGET_ROLE_ARN = BEDROCK_ROLE_NAME
TARGET_ACCOUNT_ID =  boto3.client('sts').get_caller_identity().get('Account')

AWS_REGION = "us-west-2" 
MANTLE_ENDPOINT = "https://bedrock-mantle.us-west-2.api.aws"

from aws_bedrock_token_generator import provide_token

ST_BEDROCK_API_KEY = provide_token(region="us-west-2")
print(f"Token: {ST_BEDROCK_API_KEY}")

# Fine-tuning configuration
MODEL_ID = "qwen.qwen3-32b-v1:0"  # Change to your model
TRAINING_FILE_PATH = "rft_train_data.jsonl"  # Training data file

print(f"Target Role: {TARGET_ROLE_ARN}")
print(f"Account ID: {TARGET_ACCOUNT_ID}")
print(f"Region: {AWS_REGION}")
print(f"Endpoint: {MANTLE_ENDPOINT}")
print(f"Model: {MODEL_ID}")
print(f"Training File: {TRAINING_FILE_PATH}")

---
## Step 3: Create OpenAI Client with Bedrock API Key Authentication

In [ ]:
from openai import OpenAI
import urllib3
import json
from typing import Optional

def create_openai_client(endpoint: str,
                        region: str,
                        account_id: Optional[str] = None,
                        verify_ssl: bool = False) -> OpenAI:
    """
    Create OpenAI client with custom SigV4-signing transport.
    """

    return OpenAI(
        base_url=f"{endpoint}/v1",
        api_key=ST_BEDROCK_API_KEY,  # Required by SDK but not used with SigV4
    )

# Suppress SSL warnings if not verifying SSL
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Create the client
client = create_openai_client(
    endpoint=MANTLE_ENDPOINT,
    region=AWS_REGION,
    account_id=TARGET_ACCOUNT_ID,
    verify_ssl=False  # Set to True for production
)

print("✅ OpenAI client created successfully!")
print(f"   Base URL: {MANTLE_ENDPOINT}/v1")
print(f"   Account ID: {TARGET_ACCOUNT_ID}")
print(f"   Region: {AWS_REGION}")

---
# Step 4: Learn about OpenAI compatible Fine-Tuning API Operations
---

## API Operation 1: List Fine-Tuning Jobs

In [ ]:
print("="*80)
print("📋 API Operation 1: List Fine-Tuning Jobs")
print("="*80)
print()

# List fine-tuning jobs
response = client.fine_tuning.jobs.list(limit=10)

# Print raw response
print(json.dumps(response.model_dump(), indent=2))
print()
print("List Fine Tune job call completed")

## API Operation 2: File Operations

### 2.a. Upload Files

In [ ]:
print("="*80)
print("📤 API Operation 2: Upload Training File")
print("="*80)
print()
print(f"Upload training file: {TRAINING_FILE_PATH}")
print()

# Upload training file
with open(TRAINING_FILE_PATH, 'rb') as f:
    file_response = client.files.create(
        file=f,
        purpose='fine-tune'
    )

# Print raw response
print(json.dumps(file_response.model_dump(), indent=2))
print()

# Store file ID for next steps
training_file_id = file_response.id
print(f"✅ Training file uploaded successfully: {training_file_id}")
print()

### 2.b. List files

In [ ]:
print("="*80)
print("📁 List Files")
print("="*80)
print()

# List files
files_response = client.files.list(purpose='fine-tune')

# Print raw response
print(json.dumps(files_response.model_dump(), indent=2))
print()

### 2.c. Retrieve file details

In [ ]:
print("="*80)
print("📄 Retrieve File Details")
print("="*80)
print()
print(f"Get details about file: {training_file_id}")
print()

# Retrieve file details
file_details = client.files.retrieve(training_file_id)

# Print raw response
print(json.dumps(file_details.model_dump(), indent=2))
print()

### 2.d. Delete file

In [ ]:
# Uncomment to delete a file
#delete_response = client.files.delete("14c68926-ecdb-4d13-aad8-8076208fdbfa")
#print(json.dumps(delete_response.model_dump(), indent=2))

import warnings
warnings.warn("This code snippet is provided for educational purposes. Here, we will continue the training without deleting the file")

## API Operation 3: Create Fine-Tuning Job (RFT)

**Note**: Update the Lambda ARN with your actual Lambda function for RFT jobs.
For Supervised Fine-Tuning (SFT), omit the `extra_body` parameter.

In [ ]:
print("="*80)
print("🏗️  API Operation 3: Create Fine-Tuning Job (RFT)")
print("="*80)
print()

# Create fine-tuning job with RFT method
job_response = client.fine_tuning.jobs.create(
    model=MODEL_ID,
    training_file=training_file_id,
    # Suffix field is not supported so commenting for now.
    # suffix="rft-example",  # Optional: suffix for fine-tuned model name
    extra_body={
        "method": {
            "type": "reinforcement",  # Use "supervised" for SFT
            "reinforcement": {
                "grader": {
                    "type": "lambda",
                    "lambda": {
                        "function": lambda_arn  # Replace with your Lambda ARN
                    }
                },
                "hyperparameters": {
                    "n_epochs": 1,  # Number of training epochs
                    "batch_size": 4,  # Batch size
                    "learning_rate_multiplier": 1.0  # Learning rate multiplier
                }
            }
        }
    }
)

# Print raw response
print(json.dumps(job_response.model_dump(), indent=2))
print()

# Store job ID for next steps
job_id = job_response.id
print(f"✅ Fine-tuning job created successfully: {job_id}")
print()

## API Operation 4: List Jobs with Pagination

In [ ]:
print("="*80)
print("📋 API Operation 4: List Jobs (Filtered)")
print("="*80)
print()

# List jobs with limit and pagination
response = client.fine_tuning.jobs.list(
    limit=20  # Maximum number of jobs to return
)

# Print raw response
print(json.dumps(response.model_dump(), indent=2))
print()

## API Operation 5: Describe Specific Job

In [ ]:
print("="*80)
print("🔍 API Operation 5: Describe Specific Job")
print("="*80)
print()
print(f"Get detailed information about job: {job_id}")
print()

# Retrieve specific job details
job_details = client.fine_tuning.jobs.retrieve(job_id)

# Print raw response
print(json.dumps(job_details.model_dump(), indent=2))
print(f"RFT job is currently: {job_details.status}")

## API Operation 6: List Events

In [ ]:
print("="*80)
print("📊 API Operation 6: List Events")
print("="*80)
print()
print(f"List all events for job: {job_id}")
print()

# List events for the fine-tuning job
events_response = client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=job_id,
    limit=100  # Maximum number of events to return
)

# Print raw response
print(json.dumps(events_response.model_dump(), indent=2))
print()

## Plot metrics from job

In [ ]:
import matplotlib.pyplot as plt                                                                                                                

data = events_response                                                                                                                                   
# Extract metric events                                                                                                                        
metrics = [e.data for e in events_response.data if e.type == "metrics"]                                                                        
   
steps = [m["step"] for m in metrics]                                                                                                           
fields = ["critic_rewards_mean", "actor_pg_loss", "actor_entropy",
        "actor_grad_norm", "critic_advantages_mean", "response_length_mean"]

fig, axes = plt.subplots(3, 2, figsize=(14, 10))
fig.suptitle("Fine-Tuning Metrics", fontsize=14)

for ax, field in zip(axes.flat, fields):
  values = [m[field] for m in metrics]
  ax.plot(steps, values, linewidth=1.2)
  ax.set_title(field)
  ax.set_xlabel("Step")
  ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()



### How to read these metrics
Here we basically plot the "metrics" parts of the emitted events from the job. An example of a metric is shown below:


```
{
      "id": "ftevent-c3c14785-4a3b-4dab-99a5-a15aeb6c0742",
      "created_at": 1771442218,
      "level": "info",
      "message": "Step 4/67: training metrics",
      "object": "fine_tuning.job.event",
      "data": {
        "total_steps": 67,
        "actor_grad_norm": 0.0008667297661304474,
        "response_length_mean": 519.09375,
        "step": 4,
        "actor_pg_loss": 0.10153239965438844,
        "critic_rewards_mean": 0.4375,
        "actor_entropy": 0.6235736012458801,
        "critic_advantages_mean": 0.013622610829770563
      },
      "type": "metrics"
```

Let's discuss what these mean:

| Metric | Meaning |                                                                                                                                          
  |---|---|                                                                                                                     
  | **step** / **total_steps** | Current training step / out of total  |                                                                                 
  | **critic_rewards_mean** | Average reward score across the batch (0.4375 means ~44% of responses got correct answers from your grader). This is the primary metric to watch — you want it trending up. |                                                                                                                  
  | **actor_pg_loss** | Policy gradient loss. This is the objective being optimized — how much the model's policy is being pushed toward higher-reward responses. Fluctuates naturally; no single "good" value. |
  | **actor_entropy** | How spread out the model's token probability distribution is. Higher = more exploratory/diverse outputs. If it collapses toward 0, the model is becoming too deterministic (mode collapse). You want it to decrease gradually, not crash. |
  | **actor_grad_norm** | Magnitude of the gradient update to the actor (the model). Large spikes can indicate training instability. Yours is very small (0.0009), which suggests stable, conservative updates. |
  | **critic_advantages_mean** | Average advantage estimate — how much better/worse a response was compared to the critic's baseline prediction. Near-zero (0.014) means the critic is well-calibrated. Large positive values mean the model is doing much better than expected; large negative means worse. |
  | **response_length_mean** | Average token length of generated responses (519). Worth monitoring — if it grows unboundedly, the model may be gaming length for reward. |

  **What to watch for during training:**
  - `critic_rewards_mean` trending upward = model is learning
  - `actor_entropy` collapsing to 0 = mode collapse (bad)
  - `actor_grad_norm` spiking = instability
  - `response_length_mean` exploding = reward hacking?


## API Operation 7: List Checkpoints

In [ ]:
print("="*80)
print("🎯 API Operation 7: List Checkpoints")
print("="*80)
print()
print(f"List all checkpoints for job: {job_id}")
print()

# List checkpoints for the fine-tuning job
try:
    checkpoints_response = client.fine_tuning.jobs.checkpoints.list(
        fine_tuning_job_id=job_id
    )
    
    # Print raw response
    print(json.dumps(checkpoints_response.model_dump(), indent=2))
    print()
    
    if checkpoints_response.data:
        print(f"✅ Found {len(checkpoints_response.data)} checkpoint(s)")
    else:
        print("ℹ️  No checkpoints available yet (job may still be running)")
    print()
    
except Exception as e:
    print(f"⚠️  Error listing checkpoints: {e}")
    print("   Note: Checkpoints are only available after the job starts training")
    print()

---
# Additional API Operations
---

## Cancel a Fine-Tuning Job

In [ ]:
# Uncomment to cancel a job
#cancel_response = client.fine_tuning.jobs.cancel(job_id)
#print(json.dumps(cancel_response.model_dump(), indent=2))

warnings.warn("To cancel a job, uncomment the code above and replace job_id")

## Run Inference with Fine-Tuned Model

Get the fine-tuned model ID

In [ ]:
job_details = client.fine_tuning.jobs.retrieve(job_id)
job_details

#### Once your job completes, you can run inference like this:"

In [ ]:
%%time
if job_details.status == 'succeeded' and job_details.fine_tuned_model:
    fine_tuned_model = job_details.fine_tuned_model
    print(f"Using fine-tuned model: {fine_tuned_model}")
    print()
    
    # Run inference
    inference_response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=[
            {"role": "user", "content": "Write a 100 word essay on Euclid's contributions."}
        ],
        max_tokens=100
    )
    
    print(json.dumps(inference_response.model_dump(), indent=2))
    print()
else:
    print(f"Job status: {job_details.status}")
    print("Job must be in 'succeeded' status to run inference")
    print()


print("Uncomment the code above to run inference after job completes.")

### Test streaming response

In [ ]:
#Test Streaming
from openai import OpenAI

example1 = """Gina chooses what she and her sister will watch on Netflix three times as often as her sister does. If her sister watches a total of 24 shows on Netflix per week, and each show is 50 minutes long, how many minutes of Netflix does Gina get to choose? Let's think step by step and output the final answer after '####'."""

example2 = """In the honey shop, the bulk price of honey is $5 per pound and the minimum spend is $40 before tax. The honey is taxed at $1 per pound. If Penny has paid $240 for honey, by how many pounds has Penny’s purchase exceed the minimum spend? Let's think step by step and output the final answer after "####"."""

from colorama import Fore, Style, init

stream = client.responses.create(
    model= fine_tuned_model, 
    # model = MODEL_ID, # Base model
    input=[
        {
            "role": "user",
            "content": example1,
        },
    ],
    stream=True,
    reasoning = {"effort":"low"}
)

for event in stream:
    # Each event has a 'type' and 'data'
    
    # print(event.type)
    if event.type in ['response.reasoning_part.added','response.reasoning_part.done']:
        print(Fore.GREEN + Style.DIM + "\n<thinking>\n")
    if event.type == 'response.reasoning_text.delta':
        print(Fore.GREEN + Style.DIM + event.delta, end="", flush=True)
    if event.type in ['response.output_text.delta',]:
        print(Fore.BLACK + Style.RESET_ALL + event.delta, end="", flush=True)

---
## Summary

This notebook showed end-to-end reinforcement fine-tuning (RFT) of GPT-OSS 20B on GSM8K math problems via Bedrock's OpenAI-compatible APIs. A quick recap of the steps we performed: we set up a Lambda-based reward function that scores model responses by extracting answers and comparing them to ground truth, then created the necessary IAM roles for both Lambda and Bedrock. Using a short-term Bedrock API key and the OpenAI SDK, we uploaded training data, kicked off an RFT job with a single epoch, and monitored its progress through events and checkpoints. Once training completed (67 steps, 4 checkpoints), we ran inference against the fine-tuned model using both chat completions and streaming with reasoning, and benchmarked it against the base model on latency and throughput.

For more information, please visit the documentation here - https://docs.aws.amazon.com/bedrock/latest/userguide/fine-tuning-openai-apis.html



# (Optional) Benchmarking base vs. fine-tuned model performance

It is important to understand how fine-tuning affects not just model accuracy but also inference performance. Reinforcement fine-tuning modifies the model's weights to favor higher-reward responses, but this can also change generation characteristics — response length, reasoning depth, and token distributions all shift. The benchmarking snippet below compares the base GPT-OSS 20B model against the fine-tuned version you just created across three key dimensions: time to first token (TTFT), output throughput (tokens per second), and total latency. This helps you evaluate whether the accuracy gains from RFT come with any inference cost trade-offs, and informs decisions about deployment readiness.

In [ ]:
import time                                                                                                                                    
import tiktoken                                      
import matplotlib.pyplot as plt                                                                                                                
import matplotlib.patches as mpatches                                                                                                          
                                                                                                                                             
enc = tiktoken.get_encoding("cl100k_base")                                                                                                     

def benchmark_model(client, model_id, prompt, label="model"):
  stream = client.responses.create(
      model=model_id,
      input=[{"role": "user", "content": prompt}],
      stream=True,
      reasoning={"effort": "low"}
  )

  start = time.perf_counter()
  ttft = None
  first_output_token_time = None
  reasoning_text = ""
  output_text = ""

  for event in stream:
      now = time.perf_counter()
      if event.type == 'response.reasoning_text.delta':
          if ttft is None:
              ttft = now - start
          reasoning_text += event.delta
      elif event.type == 'response.output_text.delta':
          if first_output_token_time is None:
              first_output_token_time = now - start
          output_text += event.delta

  end = time.perf_counter()
  total_time = end - start

  reasoning_tokens = len(enc.encode(reasoning_text))
  output_tokens = len(enc.encode(output_text))
  total_tokens = reasoning_tokens + output_tokens

  gen_duration = total_time - (ttft if ttft else 0)
  output_duration = total_time - (first_output_token_time if first_output_token_time else 0)

  result = {
      "label": label,
      "ttft": ttft or 0,
      "time_to_first_output": first_output_token_time or 0,
      "total_time": total_time,
      "reasoning_tokens": reasoning_tokens,
      "output_tokens": output_tokens,
      "total_tokens": total_tokens,
      "total_tokens_per_sec": total_tokens / gen_duration if gen_duration > 0 else 0,
      "output_tokens_per_sec": output_tokens / output_duration if output_duration > 0 else 0,
  }

  print(f"\n{'='*60}")
  print(f"  {label}")
  print(f"{'='*60}")
  print(f"  TTFT (first reasoning token):  {result['ttft']:.3f}s")
  print(f"  Time to first output token:    {result['time_to_first_output']:.3f}s")
  print(f"  Total time:                    {result['total_time']:.3f}s")
  print(f"  Reasoning tokens:              {result['reasoning_tokens']}")
  print(f"  Output tokens:                 {result['output_tokens']}")
  print(f"  Total tokens:                  {result['total_tokens']}")
  print(f"  Total tokens/sec:              {result['total_tokens_per_sec']:.1f}")
  print(f"  Output tokens/sec:             {result['output_tokens_per_sec']:.1f}")
  print(f"{'='*60}\n")

  return result

# --- Run benchmarks ---
prompt = "Write a 1000-word essay on the history and future of artificial intelligence."
base_result = benchmark_model(client, MODEL_ID, prompt, label="Base Model")
ft_result = benchmark_model(client, fine_tuned_model, prompt, label="Fine-Tuned Model")

# --- Plot ---
plt.rcParams.update({
  'font.family': 'sans-serif',
  'font.size': 11,
  'axes.spines.top': False,
  'axes.spines.right': False,
})

fig, axes = plt.subplots(1, 3, figsize=(16, 6))
fig.suptitle("Base vs Fine-Tuned Model — Inference Performance",
           fontsize=15, fontweight='bold', y=1.02)

colors = ["#2563EB", "#F97316"]
labels = ["Base", "Fine-Tuned"]

def pct_change(base, ft):
  if base == 0:
      return 0
  return ((ft - base) / base) * 100

def annotate_change(ax, base_val, ft_val, y_offset_frac=0.15, higher_is_better=False):
  change = pct_change(base_val, ft_val)
  is_improvement = (change > 0 and higher_is_better) or (change < 0 and not higher_is_better)
  color = "#16A34A" if is_improvement else "#DC2626"
  arrow_symbol = "▲" if change > 0 else "▼"
  sign = "+" if change > 0 else ""

  max_val = max(base_val, ft_val)
  y_pos = max_val + max_val * y_offset_frac

  ax.annotate(
      f"{arrow_symbol} {sign}{change:.1f}%",
      xy=(0.5, y_pos), fontsize=12, fontweight='bold',
      color=color, ha='center', va='bottom',
      bbox=dict(boxstyle="round,pad=0.3", facecolor=color, alpha=0.12,
                edgecolor=color, linewidth=1.2)
  )
  ax.plot([0, 1], [y_pos, y_pos], color=color, linewidth=1.5, alpha=0.6)
  ax.plot([0, 0], [base_val + max_val*0.01, y_pos], color=color, linewidth=1, alpha=0.4, linestyle='--')
  ax.plot([1, 1], [ft_val + max_val*0.01, y_pos], color=color, linewidth=1, alpha=0.4, linestyle='--')

# Panel 1: TTFT
vals = [base_result["ttft"], ft_result["ttft"]]
bars = axes[0].bar(labels, vals, color=colors, width=0.5, edgecolor='white', linewidth=1.5)
axes[0].set_title("Time to First Token", fontweight='bold', pad=12)
axes[0].set_ylabel("Seconds (lower is better)", fontsize=10, color="#666")
for bar, v in zip(bars, vals):
  axes[0].text(bar.get_x() + bar.get_width()/2, v + max(vals)*0.01,
               f"{v:.3f}s", ha='center', va='bottom', fontweight='bold', fontsize=11)
annotate_change(axes[0], vals[0], vals[1], higher_is_better=False)

# Panel 2: Output Tokens/sec
vals = [base_result["output_tokens_per_sec"], ft_result["output_tokens_per_sec"]]
bars = axes[1].bar(labels, vals, color=colors, width=0.5, edgecolor='white', linewidth=1.5)
axes[1].set_title("Output Throughput", fontweight='bold', pad=12)
axes[1].set_ylabel("Tokens/sec (higher is better)", fontsize=10, color="#666")
for bar, v in zip(bars, vals):
  axes[1].text(bar.get_x() + bar.get_width()/2, v + max(vals)*0.01,
               f"{v:.1f}", ha='center', va='bottom', fontweight='bold', fontsize=11)
annotate_change(axes[1], vals[0], vals[1], higher_is_better=True)

# Panel 3: Total Time
vals = [base_result["total_time"], ft_result["total_time"]]
bars = axes[2].bar(labels, vals, color=colors, width=0.5, edgecolor='white', linewidth=1.5)
axes[2].set_title("Total Latency", fontweight='bold', pad=12)
axes[2].set_ylabel("Seconds (lower is better)", fontsize=10, color="#666")
for bar, v in zip(bars, vals):
  axes[2].text(bar.get_x() + bar.get_width()/2, v + max(vals)*0.01,
               f"{v:.2f}s", ha='center', va='bottom', fontweight='bold', fontsize=11)
annotate_change(axes[2], vals[0], vals[1], higher_is_better=False)

for ax in axes:
  ymin, ymax = ax.get_ylim()
  ax.set_ylim(0, ymax * 1.4)
  ax.tick_params(axis='x', labelsize=11)

legend_elements = [mpatches.Patch(facecolor=colors[0], label='Base Model'),
                 mpatches.Patch(facecolor=colors[1], label='Fine-Tuned Model')]
fig.legend(handles=legend_elements, loc='upper left', frameon=True,
         fontsize=10, edgecolor='#ccc', bbox_to_anchor=(0.98, 0.98))

plt.tight_layout()
plt.show()